In [ ]:
from gurobipy import *
import numpy

In [ ]:
def exCB(model, where):
    
    #print("Callback", where)
    
    if where == GRB.Callback.MIPNODE:
        
        m = 0 #cardinalidad
        w = 0 
        d = 0
        
        for j in N:
            if model.cbGetNodeRel(x[j]) > 0.1:
                w += model.cbGetNodeRel(x[j])
                m += 1
                d += a[j]
        
        if d > b and w > m - 1 + 0.001:
        
            cut = LinExpr()
        
            for j in N:
                if model.cbGetNodeRel(x[j]) > 0.1:
                    cut += x[j]
                
            model.cbCut(cut <= m - 1)
            print("User cut")
    #Este usaremos nosotros, es para soluciones enteras       
    if where == GRB.Callback.MIPSOL:
        
        m = 0 #cardinalidad
        
        #De acá recuperamos la solución que esta calculando gurobi, esto es para el problema maestro
        for j in N:
            if model.cbGetSolution(x[j]) > 0.5:
                m += 1
        
        if m % 2 == 1:
        
            cut = LinExpr()
        
            for j in N:
                if model.cbGetSolution(x[j]) > 0.5:
                    cut = cut + (1-x[j])
                else:
                    cut = cut + x[j]
                
            model.cbLazy(cut >= 1)
            print("Lazy cut")

In [ ]:
n = 10
N = range(n)

numpy.random.seed(1)

a = numpy.random.randint(50,100,n)
c = numpy.random.randint(50,100,n)
b = int(sum(a)/2)

ip = Model()
ip.params.Presolve = 0
ip.params.PreCrush = 1
ip.params.LazyConstraints = 1#Poner este parámetro en el problema maestro.

x = ip.addVars(N, vtype = GRB.BINARY)

ip.addConstr(sum(a[j]*x[j] for j in N) <= b)

ip.setObjective(sum(c[j]*x[j] for j in N), GRB.MAXIMIZE)

ip.optimize(exCB)

print()

for j in N:
    print(j, x[j].x)